In [517]:
import pandas as pd
import numpy as np

In [518]:
# read NYISO generator info

file_name = '/Users/jinningwang/Documents/work/lmpdata/data/price/npcc_nyiso/generator.csv'
gen_data = pd.read_csv(file_name)

# read NPCC case info

case_name = '/Users/jinningwang/Documents/work/lmpdata/data/case/npcc.xlsx'
npcc_data = pd.read_excel(io = case_name, sheet_name = 1, header = 0)

In [519]:
print(gen_data.shape[0])
gen_data.head()

733


,Generator Name,PTID,Subzone,Zone,Latitude,Longitude
0,59TH STREET_GT_1,24138,CON ED NY CITY,N.Y.C.,40.766120,-73.996110
1,74TH STREET_GT_1,24260,CON ED NY CITY,N.Y.C.,40.763619,-73.943008
2,74TH STREET_GT_2,24261,CON ED NY CITY,N.Y.C.,40.763619,-73.943008
3,ADK HUDSON___FALLS,24011,NMPC CAPITAL,CAPITL,43.290067,-73.606067
4,ADK RESOURCE___RCVRY,23798,NMPC CAPITAL,CAPITL,43.306154,-73.591954


In [520]:
print(npcc_data.shape[0])
npcc_data.head()

140


,uid,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner
0,0,1,1,MILLSTONE PT,345.0,1.1,0.9,1.01517,0.084533,-72.165386,41.314961,1,1,1
1,1,2,1,MONTVILLE,345.0,1.1,0.9,1.01090,0.070801,-72.103714,41.428814,1,1,1
2,2,3,1,MONTVILLE,115.0,1.1,0.9,0.99170,0.069934,-72.103714,41.428814,1,1,1
3,3,4,1,MONTVILLE,345.0,1.1,0.9,1.01125,0.071567,-72.103714,41.428814,1,1,1
4,4,5,1,CARD ST,345.0,1.1,0.9,1.00618,0.041062,-72.234276,41.702294,1,1,1


In [521]:
# calc the distance between NYISO and NPCC

cls_gen = []
for no_npcc in range(npcc_data.shape[0]):
    cdist = []
    for no_nyiso in range(gen_data.shape[0]):
        d = np.sqrt((npcc_data["xcoord"].iloc[no_npcc] - gen_data["Longitude"].iloc[no_nyiso])**2 + (npcc_data["ycoord"].iloc[no_npcc] - gen_data["Latitude"].iloc[no_nyiso])**2)
        cdist.append(d)
    if min(cdist) <= 1:  # if the shortest distance > 1, nullify it
        loc = cdist.index(min(cdist))
        cls_gen.append(gen_data["PTID"].iloc[loc])
    else:
        cls_gen.append("")

In [522]:
# node name, uncomment if want to see

# print(sorted(set(cls_gen)))

In [523]:
# 58 bus not found in NYISO

npcc_data["extn_ID_ny"] = cls_gen

# remaining bus, uncomment if want to see

# npcc_data[["idx", "name"]][npcc_data["extn_ID_ny"] == ""]

npcc_data["extn_ID_ny"][npcc_data["extn_ID_ny"] == 323796] = ""
npcc_data["extn_ID_ny"][npcc_data["extn_ID_ny"] == 23530] = ""

print(npcc_data[npcc_data["extn_ID_ny"] == ""].shape[0])

60


/var/folders/06/z8ws9b2d733f7h6yc5qpn22w0000gn/T/ipykernel_33537/1113966961.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npcc_data["extn_ID_ny"][npcc_data["extn_ID_ny"] == 323796] = ""
/var/folders/06/z8ws9b2d733f7h6yc5qpn22w0000gn/T/ipykernel_33537/1113966961.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npcc_data["extn_ID_ny"][npcc_data["extn_ID_ny"] == 23530] = ""


In [524]:
# read PJM node info

node_list = '/Users/jinningwang/Documents/work/lmpdata/data/price/npcc_pjm/npcc_rmbus_in_pjm.csv'
node_data = pd.read_csv(node_list)

# merge PJM info
npcc_data = pd.merge(npcc_data, node_data[['idx', 'extn_ID_pjm']], on="idx", how='left')

In [525]:
npcc_data[npcc_data["idx"]==77]

,uid,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner,extn_ID_ny,extn_ID_pjm
76,76,77,1,BUCHANAN,345.0,1.1,0.9,1.03539,0.009526,-73.9474,41.264374,3,1,1,,NaN


In [526]:
nyd[nyd["extn_ID_ny"]==323796]

,idx,extn_ID_ny,lmp,time


In [527]:
# there are 99 bus can be found with data.

npcc_data["flag"] = npcc_data["extn_ID_ny"].astype(bool) + npcc_data["extn_ID_pjm"].fillna(0).astype(bool)
print(npcc_data["flag"].sum())

96


In [528]:
# find the closet three bus

cls_node_all = []
for row_id in range(npcc_data.shape[0]):
    
    if npcc_data["flag"].iloc[row_id]:
        cls_node_all.append("")
    else:
        cls_node_idx = []
        d_all = []
        idx_all = []
        for rid in range(npcc_data.shape[0]):
            if npcc_data["flag"].iloc[rid]:
                # row_id is the bus with no data, rid is the bus with data
                d = np.sqrt((npcc_data["xcoord"].iloc[row_id] - npcc_data["xcoord"].iloc[rid])**2 + (npcc_data["ycoord"].iloc[row_id] - npcc_data["ycoord"].iloc[rid])**2)
                d_all.append(d)
                idx_all.append(npcc_data["idx"].iloc[rid])
        dcp = d_all.copy()
        dcp.sort()
        for n in range(2):
            target_loc = d_all.index(dcp[n])
            cls_node_idx.append(idx_all[target_loc])
        cls_node_all.append(cls_node_idx)

npcc_data["cls_node"] = cls_node_all

npcc_data.head()

,uid,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner,extn_ID_ny,extn_ID_pjm,flag,cls_node
0,0,1,1,MILLSTONE PT,345.0,1.1,0.9,1.01517,0.084533,-72.165386,41.314961,1,1,1,23722,NaN,True,
1,1,2,1,MONTVILLE,345.0,1.1,0.9,1.01090,0.070801,-72.103714,41.428814,1,1,1,23722,NaN,True,
2,2,3,1,MONTVILLE,115.0,1.1,0.9,0.99170,0.069934,-72.103714,41.428814,1,1,1,23722,NaN,True,
3,3,4,1,MONTVILLE,345.0,1.1,0.9,1.01125,0.071567,-72.103714,41.428814,1,1,1,23722,NaN,True,
4,4,5,1,CARD ST,345.0,1.1,0.9,1.00618,0.041062,-72.234276,41.702294,1,1,1,23814,NaN,True,


In [529]:
# show the node info, uncomment to use

# node_list = '/Users/jinningwang/Documents/work/lmpdata/data/price/npcc_pjm/npcc_rmbus_in_pjm.csv'
# node_data = pd.read_csv(node_list)

# node_data["extn_ID_pjm"] = node_data["extn_ID_pjm"].astype(str).str[:-2]
# node_data

In [530]:
# read nyiso lmp

file_name = '/Users/jinningwang/Documents/work/lmpdata/data/price/npcc_nyiso/OASIS_Day-Ahead_Market_Generator_LBMP_JAU_FEB.csv'
ny_lmp = pd.read_csv(file_name)

ny_lmp.rename(columns={"Gen PTID": "extn_ID_ny", "Eastern Date Hour": "time", "DAM Gen LBMP": "lmp"}, inplace=True)

ny_lmp['time'] = pd.to_datetime(ny_lmp['time'], format='%Y/%m/%d %H:%M:%S')

ny_lmp.head()

# select 1/31/2021
cond = (ny_lmp['time'].dt.year == 2021) & (ny_lmp['time'].dt.month == 1) & (ny_lmp['time'].dt.day == 31)
nyd = ny_lmp[['extn_ID_ny', 'lmp', 'time']][cond].sort_values(by=["extn_ID_ny", 'time']).copy()

nyd = pd.merge(npcc_data[["idx", "extn_ID_ny"]], nyd, on="extn_ID_ny", how='left')

nyd.dropna(inplace=True)

In [531]:
ny_lmp[ny_lmp["extn_ID_ny"] == 323796] 

,time,Gen Name,extn_ID_ny,lmp,DAM Gen Losses,DAM Gen Congestion,DAM Gen Price Version


In [532]:
# read pjm lmp

file_name = '/Users/jinningwang/Documents/work/lmpdata/data/price/npcc_pjm/da_hrl_lmps-JAN_FEB.csv'
pjm_lmp = pd.read_csv(file_name)

pjm_lmp.rename(columns={"pnode_id": "extn_ID_pjm", "datetime_beginning_ept": "time", "total_lmp_da": "lmp"}, inplace=True)

pjm_lmp['time'] = pd.to_datetime(pjm_lmp['time'], format='%m/%d/%Y %I:%M:%S %p')
pjm_lmp['time'] = pjm_lmp['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
pjm_lmp['time'] = pd.to_datetime(pjm_lmp['time'], format='%Y/%m/%d %H:%M:%S')

# select 1/31/2021
cond = (pjm_lmp['time'].dt.year == 2021) & (pjm_lmp['time'].dt.month == 1) & (pjm_lmp['time'].dt.day == 31)
pjmd = pjm_lmp[['extn_ID_pjm', 'lmp', 'time']][cond].sort_values(by=["extn_ID_pjm", 'time']).copy()

pjmd = pd.merge(npcc_data[["idx", "extn_ID_pjm"]], pjmd, on="extn_ID_pjm", how='right')

In [533]:
# rearrange lmp data

lmpd = pjmd[['time']][pjmd["idx"]==6].reset_index(drop=True).copy()

# pjm
for row_id in pjmd['idx'].unique().tolist():
    df = pjmd[['time', 'lmp']][pjmd['idx']==row_id].copy()
    df.rename(columns={"lmp": f"vBus{row_id}"}, inplace=True)
    lmpd = pd.merge(lmpd, df, on="time", how='left')

# nyiso
for row_id in nyd['idx'].unique().tolist():
    df = nyd[['time', 'lmp']][nyd['idx']==row_id].copy()
    df.rename(columns={"lmp": f"vBus{row_id}"}, inplace=True)
    lmpd = pd.merge(lmpd, df, on="time", how='left')
    
lmpd.head()

,time,vBus132,vBus130,vBus88,vBus134,vBus45,vBus135,vBus14,vBus20,vBus6,...,vBus107,vBus108,vBus110,vBus124,vBus125,vBus126,vBus136,vBus137,vBus138,vBus140
0,2021-01-31 00:00:00,25.74,25.52,26.78,36.27,22.49,27.20,22.94,13.11,27.03,...,23.75,23.75,23.75,80.60,80.76,80.76,26.27,26.27,36.02,26.27
1,2021-01-31 01:00:00,25.11,24.87,26.04,34.81,21.88,26.31,22.27,12.72,26.28,...,21.71,21.71,21.71,74.14,74.28,74.28,23.95,23.95,32.93,23.95
2,2021-01-31 02:00:00,23.98,23.96,24.72,30.15,21.63,24.36,21.97,13.32,24.91,...,21.67,21.67,21.67,73.90,74.05,74.05,23.96,23.96,32.97,23.96
3,2021-01-31 03:00:00,22.51,22.55,23.00,24.12,21.40,21.84,21.71,13.19,23.08,...,19.52,19.52,19.52,72.97,73.07,73.07,21.70,21.70,30.87,21.70
4,2021-01-31 04:00:00,22.68,22.74,23.20,24.20,21.37,22.03,21.69,13.50,23.28,...,19.14,19.14,19.14,73.63,73.74,73.74,21.31,21.31,30.60,21.31


In [534]:
# combine lmp

npcc_lmp = npcc_data[["idx", "flag", "cls_node"]].copy()
npcc_lmp.head()

for rid in npcc_data[npcc_data["flag"]==False].index:
    idx = npcc_data["idx"].iloc[rid]
    col_name = f"vBus{idx}"
    
    [idx1, idx2] = npcc_data["cls_node"].iloc[rid]
    col1 = f"vBus{idx1}"
    col2 = f"vBus{idx2}"
    
    lmpd[col_name] = lmpd[col1]*0.5 + lmpd[col2]*0.5

/var/folders/06/z8ws9b2d733f7h6yc5qpn22w0000gn/T/ipykernel_33537/3982894585.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  lmpd[col_name] = lmpd[col1]*0.5 + lmpd[col2]*0.5


In [535]:
# integrate data into npcc_out

npcc_out = pd.read_csv('/Users/jinningwang/Documents/work/lmpdata/data/case/npcc_results.csv')

In [536]:
lmpd.iloc[0:2]

,time,vBus132,vBus130,vBus88,vBus134,vBus45,vBus135,vBus14,vBus20,vBus6,...,vBus118,vBus119,vBus120,vBus121,vBus122,vBus123,vBus127,vBus128,vBus131,vBus133
0,2021-01-31 00:00:00,25.74,25.52,26.78,36.27,22.49,27.20,22.94,13.11,27.03,...,21.78,22.295,21.78,22.295,21.78,21.78,26.080,53.62,26.080,31.735
1,2021-01-31 01:00:00,25.11,24.87,26.04,34.81,21.88,26.31,22.27,12.72,26.28,...,21.20,21.670,21.20,21.670,21.20,21.20,25.395,50.03,25.395,30.560


In [540]:
npcc_agg = npcc_out.loc[npcc_out.index < 24].copy()

for rid in range(npcc_data.shape[0]):
    rid += 1
    col = f"vBus{rid}"
    npcc_agg[col] = lmpd[col]

npcc_agg.to_csv('/Users/jinningwang/Documents/work/lmpdata/data/case/npcc_lmp.csv', index=False) 

npcc_agg.head()

,Time_s_,deltaGENCLS1,deltaGENCLS2,deltaGENCLS3,deltaGENCLS4,deltaGENCLS5,deltaGENCLS6,deltaGENCLS7,deltaGENCLS8,deltaGENCLS9,...,W_yIEEEX115,W_yIEEEX116,W_yIEEEX117,W_yIEEEX118,W_yIEEEX119,W_yIEEEX120,W_yIEEEX121,W_yIEEEX122,W_yIEEEX123,W_yIEEEX124
0,0.000000,0.465142,0.291653,0.235877,0.222933,0.202047,0.088103,0.338032,0.404395,0.353749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.033333,0.465142,0.291653,0.235877,0.222933,0.202047,0.088103,0.338032,0.404395,0.353749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.066667,0.465142,0.291653,0.235877,0.222933,0.202047,0.088103,0.338032,0.404395,0.353749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.100000,0.465142,0.291653,0.235877,0.222933,0.202047,0.088103,0.338032,0.404395,0.353749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.133333,0.465142,0.291653,0.235877,0.222933,0.202047,0.088103,0.338032,0.404395,0.353749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
